# Interactive molecular dynamics

BioSimSpace is a great tool for playing around with molecular simulations directly and interacting with them in real-time. In this notebook you'll learn how to use BioSimSpace to set up and run an equilibration protocol, then query the running process for information, plot graphs of the latest data, visualise molecular configurations, and analyse trajectory data.

Before we get started, let's import BioSimSpace so that it's available inside of our notebook.

In [ ]:
import BioSimSpace as BSS

## Creating a molecular system

First of all we need to load a molecular system.

In [ ]:
system = BSS.IO.readMolecules(["amber/ala/ala.crd", "amber/ala/ala.top"])

We have now created a molecular system. The system consists of an alanine dipeptide molecule in a box of water. To show the number of molecules in the system, run:

In [ ]:
system.nMolecules()

## Defining a simulation protocol

BioSimSpace provides functionality for defining various simulation protocols. In this notebook we will construct a typical simulation workflow that uses a sequence of simple protocols, with the output of one forming the input of the next:

1. _Minimisation:_ Energy minimisation the molecular system.
2. _Equilibration:_ Equilibration of the system to a target temperature.
3. _Production:_ Regular molecular dynamics, run at fixed temperature.
4. _FreeEnergy:_ A protocol for an individual leg of a free-energy perturbation simulation.
5. _Metadynamics:_ Metadynamics simulation to sample free energy as a function of collective variable(s). 
6. _Custom:_ A user defined protocol, e.g. a config file for a molecular dynamics package.

When defining a protocol we are configuring the type of simulation that we wish to run, as well as any options for the particular simulation. For example, to create a default equilibration protocol:

```python
protocol = BSS.Protcol.Equilibration()
```

This defines a 0.2 nanosecond equilibration protocol at a temperature of 300 Kelvin. For convenience, let's reduce the runtime. We'll also perform a heating protocol and will restrain the position of atoms in the backbone.

In [ ]:
# Initialise a short equilibration protocol.
protocol = BSS.Protocol.Equilibration(runtime=0.05*BSS.Units.Time.nanosecond,
                                      temperature_start=0*BSS.Units.Temperature.kelvin,
                                      temperature_end=300*BSS.Units.Temperature.kelvin,
                                      restrain_backbone=True)

## Initialising a process

We now have everything that is needed to create a process object. To do so, run:

In [ ]:
process = BSS.MD.run(system, protocol)

On creation, BioSimSpace searches your `PATH` for an appropriate executable for running the process. The executable that is chosen may be dependent on the available hardware and type of protocol. If a suitable executable was found, BioSimSpace will automaticall set up everthing for you, start the simulation automatically, and return a handle to the running process.

To see if the process is running:

In [ ]:
process.isRunning()

To see how many minutes the process has been running for:

In [ ]:
process.runTime()

We can also query the total energy of the molecular system (in kcal/mol):

In [ ]:
process.getTotalEnergy()

We can monitor the time, temperature, and energy as the process runs. If you run this multiple times using "CTRL+Return" you'll see the temperature slowly increasing.

In [ ]:
print(process.getTime(), process.getTemperature(), process.getTotalEnergy())

It's possible to query many other thermodynamic records. What's available depends on type of protocol and the program that is used to run the protocol.

## Plotting time series data

As well as querying the most recent records we can also get a time series of results by passing the `time_series` keyword argument to any of the data record getter methods, e.g.

```python
# Get a time series of pressure records.
pressure = process.getPressure(time_series=True)
```

BioSimSpace comes with several useful tools that are available when working inside of a Jupyter notebook. One of this is the `plot` function, that allows us to create simple x/y plot of time series data.

Let's grab the same record data as above and use it to make some graphs of the data.

In [ ]:
# Generate a plot of time vs temperature.
plot1 = BSS.Notebook.plot(process.getTime(time_series=True),
    process.getTemperature(time_series=True))

# Generate a plot of time vs energy.
plot2 = BSS.Notebook.plot(process.getTime(time_series=True),
    process.getTotalEnergy(time_series=True))

Re-run the cell using "CTRL+Return" to see the graphs update as the simulation progesses.

Being able to query a process in real time is an incredibly useful tool. This could enable us to check for convergence, or spot errors in the simulation. If you ever need to kill a running process (perhaps it was configured incorrectly), run:

```python
process.kill()
```

## Visualising the molecular system

Another useful tool that is available when working inside of a notebook. One of these is the `View` class that can be used to visualise the molecular system while a process is running. To create a `View` object we must attach it to a process (or a molecular system), e.g.:

In [ ]:
view = BSS.Notebook.View(process)

We can now visualise the system:

In [ ]:
view.system()

## Reading and analysis trajectory data

BioSimSpace comes with a set of tools for reading and analysis trajectory files. Files can be loaded directly, or if supported, can be read from a running process.

For example, to get the trajectory from the process, run:

In [ ]:
traj = process.getTrajectory()

(If you get an error, then the trajectory file may be in the process of being written. Simply try again.)

To get the current number of frames:

In [ ]:
traj.nFrames()

To get the frames as a list of system objects:

In [ ]:
frames = traj.getFrames()

The `Trajectory` class provides wrappers around some basic MDTraj analysis tools, allowing the user to compute quantities such as the root mean squared displacement (RMSD).

Let's measure the RMSD of the alanine-dipeptide molecule with a reference to its configuration in the first trajectory frame. To extract the alanine-dipeptide, we search the system for a residue named `ALA`. We'll also plot the RMSD for each frame of the trajectory.

In [ ]:
# Search the system for a residue named ALA. Since there is a single match, we take the first result.
residue = system.search("resname ALA")[0]

# Get the indices of the atoms in the molecule, relative to the original system.
indices = [system.getIndex(x) for x in residue.getAtoms()]

# Compute the RMSD for each frame and plot the result.
BSS.Notebook.plot(y=process.getTrajectory().rmsd(frame=0, atoms=indices), xlabel="Frame", ylabel="RMSD")